In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_comment_count(video_url):
    try:
        # Send a request to the YouTube video page
        response = requests.get(video_url)
        if response.status_code != 200:
            raise Exception('Failed to load page')

        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Search for the initial data using a regex pattern
        script_tags = soup.find_all('script')
        for script in script_tags:
            if script.string and 'var ytInitialData' in script.string:
                # Extract the JSON data embedded in the script tag
                match = re.search(r'var ytInitialData = ({.*?});', script.string)
                if match:
                    json_text = match.group(1)
                    json_data = json.loads(json_text)

                    # Navigate the JSON data to find the comment count
                    try:
                        comment_count = json_data['contents']['twoColumnWatchNextResults']['results']['results']['contents'][2]['itemSectionRenderer']['contents'][0]['commentsEntryPointHeaderRenderer']['commentCount']['simpleText']
                        return comment_count
                    except KeyError:
                        return None  # Return None if comment count is not found

        return None  # Return None if ytInitialData is not found
    except Exception as e:
        return f"Error: {e}"

def convert_comment_count(comment_count):
    if comment_count is None:
        return 0  # Return 0 if comment count is None
    
    try:
        if isinstance(comment_count, str):
            comment_count = comment_count.replace(',', '')  # Remove commas
            if 'K' in comment_count:
                return int(float(comment_count.replace('K', '')) * 1000)
            elif 'M' in comment_count:
                return int(float(comment_count.replace('M', '')) * 1000000)
            else:
                return int(comment_count)
        return int(comment_count)
    except ValueError as e:
        print(f"ValueError: {e} for comment_count: {comment_count}")
        return 0  # Return 0 on conversion error

def fetch_comment_count(video_code):
    video_url = f"https://www.youtube.com/watch?v={video_code}"
    comment_count = get_comment_count(video_url)
    return convert_comment_count(comment_count)

# Read the Excel file
excel_file = 'ted _data.xlsx'
sheet_name = 'Sheet1'
df = pd.read_excel(excel_file, sheet_name=sheet_name)

# Extract all video links
video_links = df['youtube_video_code'].tolist()

# Initialize a list to hold comment counts
comment_counts = []

# Use ThreadPoolExecutor to fetch comment counts concurrently
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(fetch_comment_count, video_code): video_code for video_code in video_links}
    for future in as_completed(futures):
        video_code = futures[future]
        try:
            comment_count = future.result()
            comment_counts.append(comment_count)
        except Exception as e:
            comment_counts.append(f"Error: {e}")

# Add the comment counts to the DataFrame
df['comment_count'] = comment_counts

# Print the DataFrame with the comment counts
print(df[['youtube_video_code', 'comment_count']])

# Save the updated DataFrame to a new Excel file
df.to_excel('ted_data_with_comments.xlsx', index=False)

print("Comment counts added and saved to 'ted_data_with_comments.xlsx'.")


ValueError: invalid literal for int() with base 10: 'Error: list index out of range' for comment_count: Error: list index out of range
     youtube_video_code  comment_count
0           hO8OldidMmE              0
1           gR4UlNoOrlc            369
2           5XnR-w682bw            185
3           5APf07w4J2c             37
4           rD1Rn6vai5Q            320
...                 ...            ...
4010        BS-LyKorcq8            131
4011        Sp7ZjH2IHJ8          18000
4012        cdZZpaB2kDM             22
4013        9w0PL2_-oAE              0
4014        AxWjpyRYSmw             92

[4015 rows x 2 columns]
Comment counts added and saved to 'ted_data_with_comments.xlsx'.
